# **Database initialization and ML using TF-IDF Vectorization**


This Notebook, *Notebook 1*, efficiently processes and analyzes the Yelp dataset using Pandas for data manipulation and SQLAlchemy for database interactions. Data is read from CSV files into DataFrames, encompassing Yelp's businesses, reviews, users, tips, and check-ins. The notebook features data cleaning by removing unnecessary columns and duplicates, enhancing data integrity and storage efficiency.



Key functionalities include creating a PostgreSQL database table for Illinois data to demonstrate localized analysis, and backing up the database using pg_dump for security. The notebook extends its capabilities by integrating TextBlob for sentiment analysis and applying machine learning techniques, enriching the dataset with insights into user sentiment and trends. This streamlined approach highlights the notebook's ability to handle advanced data processing and analytics within a cohesive data management framework.

**Note on Dataset Limitation:**

Due to the extensive size of the complete Yelp dataset, which requires substantial time and computing resources to process, our analysis has been strategically narrowed to focus solely on data from Illinois (IL). This approach allows us to manage computational demands effectively while providing a detailed example of localized data analysis.

**Note on Machine Learning Model Implementation:**

While there are opportunities for enhancing the machine learning model through advanced algorithms, improved feature engineering, and more sophisticated techniques, the current model serves primarily as a demonstrative tool. Its purpose is to showcase how data stored in PostgreSQL can be effectively utilized for analysis in a database management class. This implementation focuses on integrating database operations with basic machine learning to provide a practical understanding of data-driven decision.

**Importing necessary libraries**

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import subprocess
from textblob import TextBlob
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.utils import resample
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold, cross_val_score

**Pre-processing and loading of data to PostgreSQL**

In [ ]:
# Function to load JSON files
def load_json_to_df(filepath):
    """
    Load JSON data from a file into a pandas DataFrame in chunks.

    Parameters:
    - filepath: Path to the JSON file.

    Returns:
    - DataFrame containing the combined chunks.
    """
    chunks = pd.read_json(filepath, lines=True, chunksize=10000)
    df_list = []
    for chunk in chunks:
        df_list.append(chunk)
    return pd.concat(df_list, ignore_index=True)

# Load JSON files directly from current directory
df_review = load_json_to_df('review.json')
df_user = load_json_to_df('user.json')
df_business = load_json_to_df('business.json')
df_tip = load_json_to_df('tip.json')
df_checkin = load_json_to_df('checkin.json')

# Function to export df to CSV
def export_dataframe_to_csv(df, filename):
    """
    Export a DataFrame to a CSV file.
    """
    df.to_csv(filename, index=False)

# Function to load df to PostgreSQL
def load_dataframe_to_postgresql(df, engine, table_name, chunksize=5000):
    """
    Load a pandas DataFrame into a PostgreSQL table.
    """
    df.to_sql(table_name, engine, if_exists='append', index=False, chunksize=chunksize)


# Database credentials and connection
username = 'team1'
password = 'yelpdata'
host = 'localhost'
port = '5434'
database = 'yelpdb'
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

# Load CSV data into DataFrames
df_business = pd.read_csv('business.csv', on_bad_lines='skip')
df_review = pd.read_csv('review.csv', on_bad_lines='skip')
df_user = pd.read_csv('user.csv', on_bad_lines='skip')
df_tip = pd.read_csv('tip.csv', on_bad_lines='skip')
df_checkin = pd.read_csv('checkin.csv', on_bad_lines='skip')

# Data manipulation: Remove "date" columns and duplicates
if 'date' in df_review.columns:
    df_review.drop(columns=['date'], inplace=True)
df_review.drop_duplicates(inplace=True)

if 'yelping_since' in df_user.columns:
    df_user.drop(columns=['yelping_since'], inplace=True)
df_user.drop_duplicates(inplace=True)

if 'date' in df_tip.columns:
    df_tip.drop(columns=['date'], inplace=True)
df_tip.drop_duplicates(inplace=True)

df_business.drop_duplicates(inplace=True)
df_checkin.drop_duplicates(inplace=True)

# Load the DataFrame into PostgreSQL
load_dataframe_to_postgresql(df_business, engine, 'business')
load_dataframe_to_postgresql(df_review, engine, 'review')
load_dataframe_to_postgresql(df_user, engine, 'user')
load_dataframe_to_postgresql(df_tip, engine, 'tip')
load_dataframe_to_postgresql(df_checkin, engine, 'checkin')

# Backup the database
subprocess.run([
    'pg_dump',
    '-U', username,
    '-W',
    '-F', 'c',
    '-d', database,
    '-f', 'yelpdb.dump'
], check=True)

print("Export and Backup completed")

Export and Backup completed


**Creating a table in SQL for Illinois (IL) data, joining business_id from business and review data**

In [ ]:
# Database credentials
db_username = 'postgres'
db_password = 'yelpdata'
db_host = 'localhost'
db_port = '5434'
db_name = 'yelpdb'

# Create a connection to the database
engine = create_engine(f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# Define the SQL query
create_table_query = text("""
CREATE TABLE IF NOT EXISTS IL_data AS
SELECT r.review_id, r.user_id, r.business_id, r.stars, r.text,
       b.name AS business_name, b.address AS business_address, b.city AS business_city,
       b.state AS business_state, b.categories AS business_categories,
       u.name AS user_name
FROM reviews r
JOIN businesses b ON r.business_id = b.business_id
JOIN users u ON r.user_id = u.user_id
WHERE b.state = 'IL';
""")

# Execute the query to create a new table
with engine.begin() as conn:  # auto-commit at the end of the block
    conn.execute(create_table_query)

print("Table IL_data created successfully.")

Table IL_data created successfully.


**Normalization and Indexing**

In [ ]:
# Normalization: create separate tables for businesses and users

# Create businesses table
with engine.begin() as conn:
    conn.execute(text("""
    CREATE TABLE IF NOT EXISTS businesses (
        business_id VARCHAR PRIMARY KEY,
        name VARCHAR,
        address VARCHAR,
        city VARCHAR,
        state VARCHAR,
        categories VARCHAR
    );
    """))

    conn.execute(text("""
    INSERT INTO businesses (business_id, name, address, city, state, categories)
    SELECT DISTINCT business_id, business_name, business_address, business_city,
           business_state, business_categories
    FROM IL_data;
    """))

# Create users table
# Insert into users table, ignoring duplicates
with engine.begin() as conn:
    conn.execute(text("""
    INSERT INTO users (user_id, name)
    SELECT DISTINCT user_id, user_name
    FROM IL_data
    ON CONFLICT (user_id) DO NOTHING;
    """))

# Indexing

# Create indexes for performance improvement
with engine.begin() as conn:
    conn.execute(text("CREATE INDEX IF NOT EXISTS idx_business_state ON businesses (state);"))
    conn.execute(text("CREATE INDEX IF NOT EXISTS idx_user_id ON reviews (user_id);"))

# Prepare for full-text search on the reviews table
with engine.begin() as conn:
    conn.execute(text("""
    ALTER TABLE reviews
    ADD COLUMN IF NOT EXISTS review_text_tsvector tsvector
    GENERATED ALWAYS AS (to_tsvector('english', text)) STORED;
    """))

    conn.execute(text("CREATE INDEX IF NOT EXISTS idx_review_text ON reviews USING GIN(review_text_tsvector);"))

print("Normalization and indexing complete.")


Normalization and indexing complete.


**Example Query of Business and Users Table**

In [ ]:
# Fetch the first few rows from the businesses table
query_businesses = "SELECT * FROM businesses LIMIT 5;"
businesses_head = pd.read_sql_query(query_businesses, engine)
print("First 5 entries in businesses table:")
print(businesses_head)

# Fetch the first few rows from the users table
query_users = "SELECT * FROM users LIMIT 5;"
users_head = pd.read_sql_query(query_users, engine)
print("\nFirst 5 entries in users table:")
print(users_head)

First 5 entries in businesses table:
              business_id                      name  \
0  Pns2l4eNsfO8kk83dixA6A  Abby Rappoport, LAC, CMQ   
1  mpf3x-BjTdTEA3yCZrAYPw             The UPS Store   
2  tUFrWirKiKi_TAnsVWINQQ                    Target   
3  MTSW4McQd7CbVtyjqoe9mw        St Honore Pastries   
4  mWMc6_wTdE0EUBKIGXDVfA  Perkiomen Valley Brewery   

                           address           city state postal_code  \
0           1616 Chapala St, Ste 2  Santa Barbara    CA       93101   
1  87 Grasso Plaza Shopping Center         Affton    MO       63123   
2             5255 E Broadway Blvd         Tucson    AZ       85711   
3                      935 Race St   Philadelphia    PA       19107   
4                    101 Walnut St     Green Lane    PA       18054   

    latitude   longitude  stars  review_count  is_open  \
0  34.426679 -119.711197    5.0             7        0   
1  38.551126  -90.335695    3.0            15        1   
2  32.223236 -110.880452    3.5

**Query displaying created IL Table**

In [ ]:
# SQL query to fetch the first few rows
fetch_query = "SELECT * FROM IL_data;"

# Execute the query and load the data into a DataFrame
IL_data = pd.read_sql_query(fetch_query, engine)

# Display the first few rows of the DataFrame
print(IL_data.head())

                review_id                 user_id             business_id  \
0  X9jSqBSMXWgHdmbnwYosvg  wqpRODS7g9lKEncUTJpJ7A  BGiQRikW27X8wfIE71iQBw   
1  Bgk1IFHpU7t1lLAz03Ej3g  rFXFdtECXKW2mvR5qcFiow  U4ZXDA_3gIRKj6zKhelYoQ   
2  Bgk1IFHpU7t1lLAz03Ej3g  rFXFdtECXKW2mvR5qcFiow  U4ZXDA_3gIRKj6zKhelYoQ   
3  Bgk1IFHpU7t1lLAz03Ej3g  rFXFdtECXKW2mvR5qcFiow  U4ZXDA_3gIRKj6zKhelYoQ   
4  5jxbkBMI9r9GT8PJ7bWC-g  vDVmry9zqbl8t4QtHyR5hA  16QLhabqmknpdVm8WwERQg   

   stars                                               text  \
0    1.0  I recently ordered 2 pizzas, wings, and pasta ...   
1    5.0  My family and I just moved to the area not too...   
2    5.0  My family and I just moved to the area not too...   
3    5.0  My family and I just moved to the area not too...   
4    5.0  My husband and I tried this place for the firs...   

          business_name    business_address business_city business_state  \
0             Pizza Hut      608 N Bluff Rd  Collinsville             IL   
1  Pin

**Sentiment and Quality Analysis of IL Reviews**

In [ ]:
# Function to calculate sentiment extremity of reviews
def calculate_sentiment_extremity(text, stars):
    """
    Calculate sentiment polarity and subjectivity using TextBlob and categorize based on extremity.

    Args:
        text (str): The review text.
        stars (int): The star rating of the review.

    Returns:
        int: 1 if sentiment is extreme or the text is overly simplistic for a 5-star review, 0 otherwise.
    """
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity
    subjectivity = analysis.sentiment.subjectivity

    # Check for extreme positive/negative and subjective sentiments, or simplistic 5-star reviews
    if ((polarity > 0.8 or polarity < -0.8) and subjectivity > 0.5) or (stars == 5 and len(text.split()) < 5):
        return 1
    return 0

# Function to calculate review quality
def check_review_quality(text):
    """
    Check for various indicators of poor quality in the review text.

    Args:
        text (str): The review text.

    Returns:
        int: 1 if the text shows signs of poor quality, 0 otherwise.
    """
    # Very short text
    if len(text) < 50:
        return 1
    # High level of word repetition
    elif len(set(text.split())) < len(text.split()) * 0.7:
        return 1
    # Highly subjective text might be less reliable
    elif TextBlob(text).sentiment.subjectivity > 0.8:
        return 1
    # Excessive punctuation used
    elif bool(re.search(r'[\!\?]{2,}', text)):
        return 1
    return 0

# Apply refined sentiment analysis considering stars
IL_data['extreme_sentiment'] = IL_data.apply(
    lambda x: calculate_sentiment_extremity(x['text'], x['stars']), axis=1
)

# Apply improved quality checks
IL_data['poor_quality'] = IL_data['text'].apply(check_review_quality)

# Adjust the criteria for likely fake reviews
IL_data['likely_fake'] = IL_data[['extreme_sentiment', 'poor_quality']].sum(axis=1)
IL_data['likely_fake'] = IL_data['likely_fake'].apply(lambda x: 1 if x >= 2 else 0)

# Display the outcome
print(IL_data.head())
print(IL_data['likely_fake'].value_counts())


                review_id                 user_id             business_id  \
0  X9jSqBSMXWgHdmbnwYosvg  wqpRODS7g9lKEncUTJpJ7A  BGiQRikW27X8wfIE71iQBw   
1  Bgk1IFHpU7t1lLAz03Ej3g  rFXFdtECXKW2mvR5qcFiow  U4ZXDA_3gIRKj6zKhelYoQ   
2  Bgk1IFHpU7t1lLAz03Ej3g  rFXFdtECXKW2mvR5qcFiow  U4ZXDA_3gIRKj6zKhelYoQ   
3  Bgk1IFHpU7t1lLAz03Ej3g  rFXFdtECXKW2mvR5qcFiow  U4ZXDA_3gIRKj6zKhelYoQ   
4  5jxbkBMI9r9GT8PJ7bWC-g  vDVmry9zqbl8t4QtHyR5hA  16QLhabqmknpdVm8WwERQg   

   stars                                               text  \
0    1.0  I recently ordered 2 pizzas, wings, and pasta ...   
1    5.0  My family and I just moved to the area not too...   
2    5.0  My family and I just moved to the area not too...   
3    5.0  My family and I just moved to the area not too...   
4    5.0  My husband and I tried this place for the firs...   

          business_name    business_address business_city business_state  \
0             Pizza Hut      608 N Bluff Rd  Collinsville             IL   
1  Pin

**Preprocessing and Data Splitting**

In [ ]:
# 'text' column to be vectorized and 'stars', 'extreme_sentiment', and 'poor_quality' are other features
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # Limiting features to avoid overfitting

# Create a transformer for combining TF-IDF vectorization and scaling other numeric features
preprocessor = ColumnTransformer(
    transformers=[
        ('txt', tfidf_vectorizer, 'text'),
        ('num', StandardScaler(), ['stars', 'extreme_sentiment', 'poor_quality'])
    ])

# Prepare features
X = preprocessor.fit_transform(IL_data)
y = IL_data['likely_fake']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Handling Imbalanced Data and Model Training**

In [ ]:
# Apply SMOTE to the training data to address class imbalance
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Train the RandomForest classifier with class weight adjustment for further balancing
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train_smote, y_train_smote)

# Predict on the test set and print the classification report
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     30867
           1       1.00      0.99      0.99       232

    accuracy                           1.00     31099
   macro avg       1.00      0.99      1.00     31099
weighted avg       1.00      1.00      1.00     31099



**Class Balancing by Downsampling**

In [ ]:
# Separate majority and minority classes
df_majority = IL_data[IL_data.likely_fake == 0]
df_minority = IL_data[IL_data.likely_fake == 1]

# Downsample majority class
df_majority_downsampled = resample(df_majority,
                                   replace=False,    # sample without replacement
                                   n_samples=len(df_minority),  # to match minority class
                                   random_state=42)  # reproducible results

# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# Display new class counts
print(df_downsampled.likely_fake.value_counts())

# Use this balanced data for model training
X = preprocessor.fit_transform(df_downsampled)
y = df_downsampled['likely_fake']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


# Separate majority and minority classes
df_majority = IL_data[IL_data['likely_fake'] == 0]
df_minority = IL_data[IL_data['likely_fake'] == 1]

# Downsample majority class without replacement to match minority class size
df_majority_downsampled = resample(df_majority,
                                   replace=False,  # Sample without replacement
                                   n_samples=len(df_minority),  # Match minority class size
                                   random_state=42)  # Ensure reproducibility

# Combine the downsampled majority class with the minority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# Display new class counts to verify balancing
print("New class counts after downsampling:")
print(df_downsampled['likely_fake'].value_counts())

# Use the balanced data for model training
X = preprocessor.fit_transform(df_downsampled)
y = df_downsampled['likely_fake']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


likely_fake
0    1146
1    1146
Name: count, dtype: int64
New class counts after downsampling:
likely_fake
0    1146
1    1146
Name: count, dtype: int64


**Cross-Validation using StatifiedKFold**

In [ ]:
# Initialize the cross-validator with 5 splits, maintaining the same class proportions
cv = StratifiedKFold(n_splits=5)

# Compute the F1-score using macro averaging across folds
scores = cross_val_score(model, X, y, cv=cv, scoring='f1_macro')

# Display the average F1-score across all folds
print("Average F1-score:", scores.mean())

Average F1-score: 0.989526784703837


**Cross-Validation using StatifiedShuffleSplit**

In [ ]:
# Initialize the stratified shuffle splitter for cross-validation
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

# Define the RandomForest classifier with balanced class weights
model = RandomForestClassifier(class_weight='balanced', random_state=42)

# Compute the F1-score using macro averaging, over random stratified splits
scores = cross_val_score(model, X, y, cv=sss, scoring='f1_macro')

# Display the revised average F1-score across splits
print("Revised average F1-score:", scores.mean())

Revised average F1-score: 0.9995642618795596
